In [11]:
# This notebook demonstrates code functionality with the modified Axelrod library. 
# Refer to repo ReadMe for details to install the modified Axelrod library.

import pd_exp
from helper_funcs import partitions
import pandas as pd
from pathlib import Path
from online_settings import player_names, stag, high_t
import time, json

This simulation was designed to collect data from multiple groups of round-robin tournaments. To keep vocabulary consistent, let me introduce some project terminology:
- Supergame: a single PD instance between two players that consists of one or more rounds of play.
- Tournament: a round-robin tournament between a select group of players. This concept can also be referred to as a team.
- System: a collection of one or more teams.

In the code below, I setup data collection for two different experiments. In experiment 1, I consider how the outcome differs for one system with one team of deterministic players in two different game variations: classic PD and Stag Hunt. 

#### Experiment 1: Comparing team performance across classic PD and Stag Hunt

In [2]:
### For experiment 1
# Since this simulation gathers data for systems, I needed to code systems as ordered-lists of 
# teams. To implement ordered-lists of teams I nested player_names within closed-brackets and then a tuple to create an ordered-list.
# The simulation code allows for processing numerous systems, so I enclose the tournament within another pair
# of closed-brackets and then a tuple.

part_list = tuple([tuple([player_names])]) # 'part' is shorthand for partition

In [3]:
print(part_list)

((['Alternator', 'Anti Tit For Tat', 'Bitter Cooperator', 'Bitter Cooperator Def', 'Bully', 'Cooperator', 'Cooperator Def', 'Coop When Both Defect', 'Coop When Both Defect 1', 'Curious Defector', 'Cycler DC', 'Defector', 'Fourteen Coop', 'Fourteen Defect', 'Grim Trigger', 'Seven Coop', 'Seven Defect', 'Stubborn Def', 'Suspicious Tit For Tat', 'Sucker Coop', 'Sucker Defect', 'Thirteen Coop', 'Thirteen Defect', 'Tit For Tat', 'Two Coop', 'Two Defect', 'Win-Shift Lose-Stay', 'Win-Shift Lose-Stay Coop', 'Win-Stay Lose-Shift', 'Win-Stay Lose-Shift Defect'],),)


In [4]:
print('Total count of systems: ',len(part_list))  #Should be 5775 for n=12 and k=4
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with at partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)


print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()


print('Saving experiment data')
### For experiment 1
path = 'Data/Experiment1/'
classic_pdExp.save_data(path, 'ClassicPD_test_DELETE')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_test_DELETE')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  1
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  (['Alternator', 'Anti Tit For Tat', 'Bitter Cooperator', 'Bitter Cooperator Def', 'Bully', 'Cooperator', 'Cooperator Def', 'Coop When Both Defect', 'Coop When Both Defect 1', 'Curious Defector', 'Cycler DC', 'Defector', 'Fourteen Coop', 'Fourteen Defect', 'Grim Trigger', 'Seven Coop', 'Seven Defect', 'Stubborn Def', 'Suspicious Tit For Tat', 'Sucker Coop', 'Sucker Defect', 'Thirteen Coop', 'Thirteen Defect', 'Tit For Tat', 'Two Coop', 'Two Defect', 'Win-Shift Lose-Stay', 'Win-Shift Lose-Stay Coop', 'Win-Stay Lose-Shift', 'Win-Stay Lose-Shift Defect'],)
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Bitter Cooperator,Bitter Cooperator Def,Bully,Coop When Both Defect,Coop When Both Defect 1,Cooperator,Cooperator Def,Curious Defector,Cycler DC,Defector,Fourteen Coop,Fourteen Defect,Grim Trigger,Seven Coop,Seven Defect,Stu

Analysing: 100%|██████████| 25/25 [00:02<00:00, 11.58it/s]

***Processing number  1
partition list:  (['Alternator', 'Anti Tit For Tat', 'Bitter Cooperator', 'Bitter Cooperator Def', 'Bully', 'Cooperator', 'Cooperator Def', 'Coop When Both Defect', 'Coop When Both Defect 1', 'Curious Defector', 'Cycler DC', 'Defector', 'Fourteen Coop', 'Fourteen Defect', 'Grim Trigger', 'Seven Coop', 'Seven Defect', 'Stubborn Def', 'Suspicious Tit For Tat', 'Sucker Coop', 'Sucker Defect', 'Thirteen Coop', 'Thirteen Defect', 'Tit For Tat', 'Two Coop', 'Two Defect', 'Win-Shift Lose-Stay', 'Win-Shift Lose-Stay Coop', 'Win-Stay Lose-Shift', 'Win-Stay Lose-Shift Defect'],)
Instantiating tournament object with these players:  Alternator,Anti Tit For Tat,Bitter Cooperator,Bitter Cooperator Def,Bully,Coop When Both Defect,Coop When Both Defect 1,Cooperator,Cooperator Def,Curious Defector,Cycler DC,Defector,Fourteen Coop,Fourteen Defect,Grim Trigger,Seven Coop,Seven Defect,Stubborn Def,Sucker Coop,Sucker Defect,Suspicious Tit For Tat,Thirteen Coop,Thirteen Defect,Tit Fo


Analysing: 100%|██████████| 25/25 [00:01<00:00, 21.91it/s]

***Processing number  1
Saving experiment data
--- 117.66995191574097 seconds ---


Experiment 1: Data Comparison Across Games

In [56]:
classic_dataf1 = pd.read_csv('Data/Experiment1/ClassicPD_test_DELETE_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf1 = pd.read_csv('Data/Experiment1/StagHunt_test_DELETE_RPST_5_1_0_3.csv', index_col=0)

In [57]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,8,9,6,7,10,11,12,13,14,15,16,17,18,20,21,19,22,23,24,25,26,27,28,29,30","Alternator,Anti Tit For Tat,Bitter Cooperator,Bitter Cooperator Def,Bully,Coop When Both Defect,Coop When Both Defect 1,Cooperator,Cooperator Def,Curious Defector,Cycler DC,Defector,Fourteen Coop,Fourteen Defect,Grim Trigger,Seven Coop,Seven Defect,Stubborn Def,Sucker Coop,Sucker Defect,Suspicious Tit For Tat,Thirteen Coop,Thirteen Defect,Tit For Tat,Two Coop,Two Defect,Win-Shift Lose-Stay,Win-Shift Lose-Stay Coop,Win-Stay Lose-Shift,Win-Stay Lose-Shift Defect",2.254122,1.377258,0.237339


In [58]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf1[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist
1,"1,2,3,4,5,8,9,6,7,10,11,12,13,14,15,16,17,18,20,21,19,22,23,24,25,26,27,28,29,30","Alternator,Anti Tit For Tat,Bitter Cooperator,Bitter Cooperator Def,Bully,Coop When Both Defect,Coop When Both Defect 1,Cooperator,Cooperator Def,Curious Defector,Cycler DC,Defector,Fourteen Coop,Fourteen Defect,Grim Trigger,Seven Coop,Seven Defect,Stubborn Def,Sucker Coop,Sucker Defect,Suspicious Tit For Tat,Thirteen Coop,Thirteen Defect,Tit For Tat,Two Coop,Two Defect,Win-Shift Lose-Stay,Win-Shift Lose-Stay Coop,Win-Stay Lose-Shift,Win-Stay Lose-Shift Defect",2.209172,1.836442,0.237339


In [55]:
# SIDENOTE: Documentation (i.e. docstring) for code can be retrieved with '?'. Here are some examples.

?pd_exp.PdExp

Init signature: pd_exp.PdExp(list_of_systems, game_type=None)
Docstring:     
A class to represent a series of systems. 

...

Attributes
----------
game : axelrod.game (object)
    container for game matrix and scoring logic
sys_list : tuple
    three-dimensional ordered list, or tuple, where the first-dimension is
    a list of systems, the 2nd-dimension is the team list, and the 
    3rd-dimension is the individual players. 

Methods
-------
run_experiments:
    Iterates through each system, runs the different tournaments and 
    then saves the data to the data attribute.
save_data(path_to_directory, descrip_name):
    Saves experiment data as a csv file
File:           ~/Development/PrisonersDilemma/pd_exp.py
Type:           type
Subclasses:     


In [6]:
?pd_exp

Type:        module
String form: <module 'pd_exp' from '/Users/jessegoodspeed1/Development/PrisonersDilemma/pd_exp.py'>
File:        ~/Development/PrisonersDilemma/pd_exp.py
Docstring:  
Pd_exp: Prisoner's Dilemma Experimentation

Generate data for numerous Prisoner's Dilemma tournaments or systems of 
tournaments.

Classes:

    PdTournament
        A class to represent a tournament.
    PdSystem 
        Generate data for multiple tournaments and organize it into a dataframe
    PdExp
        Generate data for multiple systems
        
Functions:

    grouper(iterable, n, fillvalue=None) -> iterable of n-sized-chunks
        Collect data into fixed-length chunks or blocks
    avg_normalised_state(object, tuple) -> float
        Returns the tournament average for given state distribution (e.g.
        (C,C), (D,D), (C,D), (D,C))


#### Experiment 2: Comparing systems of 3 teams across classic PD and Stag Hunt

In [7]:
### For experiment 2
# Checking for txt file with list of systems
p = Path('partition_list_12_strategies_3teams_of4_DEMO.txt')
if p.exists():
   with open(p, "r") as f:
       part_list = json.load(f)
else:
   print('creating partitions and saving to file')
   part_list = list(partitions(player_names,4))
   p.touch()
   with open(p, "w") as f:
       json.dump(part_list, f)

In [8]:
part_list = part_list[:3] # Let's just consider 3 different systems
print('Total count of systems: ',len(part_list))
start_time = time.time()

print('Instantiate PD Experiments with different game types')
# To instantiate a PD Experiment, I use the PDExp constructor with a partition list and game-type, which is 
# by default the classic PD)
classic_pdExp = pd_exp.PdExp(part_list)
stag_pdExp = pd_exp.PdExp(part_list,game_type=stag)

print('Running experiments and computing data')
classic_pdExp.run_experiments()
stag_pdExp.run_experiments()

print('Save the data')
path = 'Data/Experiment2/'
classic_pdExp.save_data(path, 'ClassicPD_4x3_12uniq_DEMO')  # CHANGE file_name string when necessary
stag_pdExp.save_data(path, 'StagHunt_4x3_12uniq_DEMO')  # CHANGE file_name string when necessary

print("--- %s seconds ---" % (time.time() - start_time))

Total count of systems:  3
Instantiate PD Experiments with different game types
Running experiments and computing data
partition list:  [['Bitter Cooperator', 'Suspicious Tit For Tat', 'Win-Stay Lose-Shift', 'Bully'], ['Cooperator', 'Alternator', 'Cycler DC', 'Defector'], ['Anti Tit For Tat', 'Grim Trigger', 'Win-Shift Lose-Stay', 'Tit For Tat']]
Instantiating tournament object with these players:  Bitter Cooperator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift


Analysing: 100%|██████████| 25/25 [00:00<00:00, 60.17it/s]

Instantiating tournament object with these players:  Alternator,Cooperator,Cycler DC,Defector



Analysing: 100%|██████████| 25/25 [00:00<00:00, 75.44it/s]

Instantiating tournament object with these players:  Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay


...

Instantiating tournament object with these players:  Bitter Cooperator,Cooperator,Cycler DC,Defector



Analysing: 100%|██████████| 25/25 [00:00<00:00, 98.00it/s]

Instantiating tournament object with these players:  Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay



Analysing: 100%|██████████| 25/25 [00:00<00:00, 64.63it/s]


***Processing number  3
Save the data
--- 818.920313835144 seconds ---


Experiment 2: Data Comparison Across Systems, Teams, and Games

In [36]:
classic_dataf = pd.read_csv('Data/Experiment2/ClassicPD_4x3_12uniq_DEMO_RPST_3_1_0_5.csv', index_col=0)
stagHunt_dataf = pd.read_csv('Data/Experiment2/StagHunt_4x3_12uniq_DEMO_RPST_5_1_0_3.csv', index_col=0)
classic_dataf = classic_dataf.reset_index(drop=True)
stagHunt_dataf = stagHunt_dataf.reset_index(drop=True)

In [37]:
classic_dataf.columns

Index(['System ID', 'SYS MIN Score', 'SYS AVG Score', 'MIN of Team Avgs',
       'AVG of Team Mins', 'SYS CC Dist AVG', 'SYS CC Dist MIN', 'Team1',
       'Player1', 'P1_Norm_Score', 'Player2', 'P2_Norm_Score', 'Player3',
       'P3_Norm_Score', 'Player4', 'P4_Norm_Score', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team2', 'Player1.1',
       'P1_Norm_Score.1', 'Player2.1', 'P2_Norm_Score.1', 'Player3.1',
       'P3_Norm_Score.1', 'Player4.1', 'P4_Norm_Score.1', 'Team2 Avg Score',
       'Team2 Min Score', 'Team2 Avg CC Dist', 'Team3', 'Player1.2',
       'P1_Norm_Score.2', 'Player2.2', 'P2_Norm_Score.2', 'Player3.2',
       'P3_Norm_Score.2', 'Player4.2', 'P4_Norm_Score.2', 'Team3 Avg Score',
       'Team3 Min Score', 'Team3 Avg CC Dist'],
      dtype='object')

In [41]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team2','Team3']])

,System ID,Team1,Team2,Team3
0,"3,5,19,29_1,6,11,12_2,15,24,27","Bitter Cooperator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift","Alternator,Cooperator,Cycler DC,Defector","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"
1,"1,3,5,29_6,11,12,19_2,15,24,27","Alternator,Bitter Cooperator,Bully,Win-Stay Lose-Shift","Cooperator,Cycler DC,Defector,Suspicious Tit For Tat","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"
2,"1,5,19,29_3,6,11,12_2,15,24,27","Alternator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bitter Cooperator,Cooperator,Cycler DC,Defector","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"


In [42]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team2','Team3']])

,System ID,Team1,Team2,Team3
0,"3,5,19,29_1,6,11,12_2,15,24,27","Bitter Cooperator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift","Alternator,Cooperator,Cycler DC,Defector","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"
1,"1,3,5,29_6,11,12,19_2,15,24,27","Alternator,Bitter Cooperator,Bully,Win-Stay Lose-Shift","Cooperator,Cycler DC,Defector,Suspicious Tit For Tat","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"
2,"1,5,19,29_3,6,11,12_2,15,24,27","Alternator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift","Bitter Cooperator,Cooperator,Cycler DC,Defector","Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay"


How do the systems compare within the same game?

In [34]:

classic_dataf.loc[:,:'SYS CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN
0,"3,5,19,29_1,6,11,12_2,15,24,27",1.00,2.397586,2.333333,1.662302,0.276521,0.166667
1,"1,3,5,29_6,11,12,19_2,15,24,27",1.25,2.366038,2.216667,1.767857,0.331680,0.208333
2,"1,5,19,29_3,6,11,12_2,15,24,27",1.50,2.361872,2.215079,1.745635,0.270569,0.201587


In [35]:

stagHunt_dataf.loc[:,:'SYS CC Dist MIN']

,System ID,SYS MIN Score,SYS AVG Score,MIN of Team Avgs,AVG of Team Mins,SYS CC Dist AVG,SYS CC Dist MIN
0,"3,5,19,29_1,6,11,12_2,15,24,27",1.666667,2.387599,2.000000,1.803704,0.276521,0.166667
1,"1,3,5,29_6,11,12,19_2,15,24,27",1.966667,2.560946,2.100000,2.211905,0.331680,0.208333
2,"1,5,19,29_3,6,11,12_2,15,24,27",1.694444,2.355853,2.076984,1.914815,0.270569,0.201587


How did team 1 and 2 performance differ across systems?

In [43]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 'display.max_colwidth', None):  # more options can be specified also
    display(classic_dataf[['System ID', 'Team1', 'Team1 Avg Score',
       'Team1 Min Score', 'Team1 Avg CC Dist', 'Team2','Team2 Avg Score',
       'Team2 Min Score', 'Team2 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team2,Team2 Avg Score,Team2 Min Score,Team2 Avg CC Dist
0,"3,5,19,29_1,6,11,12_2,15,24,27","Bitter Cooperator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.363889,2.016667,0.302778,"Alternator,Cooperator,Cycler DC,Defector",2.333333,1.00,0.166667
1,"1,3,5,29_6,11,12,19_2,15,24,27","Alternator,Bitter Cooperator,Bully,Win-Stay Lose-Shift",2.385913,2.083333,0.426587,"Cooperator,Cycler DC,Defector,Suspicious Tit For Tat",2.216667,1.25,0.208333
2,"1,5,19,29_3,6,11,12_2,15,24,27","Alternator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.215079,1.766667,0.201587,"Bitter Cooperator,Cooperator,Cycler DC,Defector",2.375000,1.50,0.250000


In [44]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf[['System ID', 'Team1', 'Team1 Avg Score',
                            'Team1 Min Score', 'Team1 Avg CC Dist', 'Team2',
                            'Team2 Avg Score', 'Team2 Min Score', 
                            'Team2 Avg CC Dist']])

,System ID,Team1,Team1 Avg Score,Team1 Min Score,Team1 Avg CC Dist,Team2,Team2 Avg Score,Team2 Min Score,Team2 Avg CC Dist
0,"3,5,19,29_1,6,11,12_2,15,24,27","Bitter Cooperator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.463889,1.694444,0.302778,"Alternator,Cooperator,Cycler DC,Defector",2.000000,1.666667,0.166667
1,"1,3,5,29_6,11,12,19_2,15,24,27","Alternator,Bitter Cooperator,Bully,Win-Stay Lose-Shift",2.883929,2.619048,0.426587,"Cooperator,Cycler DC,Defector,Suspicious Tit For Tat",2.100000,1.966667,0.208333
2,"1,5,19,29_3,6,11,12_2,15,24,27","Alternator,Bully,Suspicious Tit For Tat,Win-Stay Lose-Shift",2.076984,1.694444,0.201587,"Bitter Cooperator,Cooperator,Cycler DC,Defector",2.291667,2.000000,0.250000


How did team 3 perform across the two games being compared?

In [53]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(classic_dataf.loc[0, ['Team3', 'Team3 Avg Score',
                            'Team3 Min Score', 'Team3 Avg CC Dist']])

Team3                Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay
Team3 Avg Score                                                           2.495536
Team3 Min Score                                                           1.970238
Team3 Avg CC Dist                                                         0.360119
Name: 0, dtype: object

In [54]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None, 
                       'display.max_colwidth', None):  # more options can be 
                                                       # specified also
    display(stagHunt_dataf.loc[0, ['Team3', 'Team3 Avg Score',
                            'Team3 Min Score', 'Team3 Avg CC Dist']])

Team3                Anti Tit For Tat,Grim Trigger,Tit For Tat,Win-Shift Lose-Stay
Team3 Avg Score                                                           2.698909
Team3 Min Score                                                               2.05
Team3 Avg CC Dist                                                         0.360119
Name: 0, dtype: object